In [28]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
# Loading the cbd region data
data1 = pd.read_csv("final_cbd_data.csv")
data1 = pd.DataFrame(data1)    # converting into the data frame

In [30]:
# Loading the regional data
data2 = pd.read_csv("final_regional_data.csv")
data2 = pd.DataFrame(data2)  

In [31]:
# Combining the cbd and regional data
data= pd.concat([data1,data2])

In [32]:
# Displaying the final data
data

,Unnamed: 0,PRCP,TAVG,index,Cbd_Regional,total,left,left_avg,year,day,hour
0,2019-01-01,0.000,65.5,0,cbd,13.0,6.0,1.0,2019.0,1.0,13.0
1,2019-01-02,0.000,65.0,1,cbd,18.0,8.0,1.0,2019.0,2.0,11.0
2,2019-01-03,0.000,87.5,2,cbd,7.0,3.5,1.0,2019.0,3.0,12.0
3,2019-01-04,1.524,59.5,3,cbd,19.0,9.0,1.0,2019.0,4.0,11.0
4,2019-01-05,0.000,65.5,4,cbd,17.0,8.0,1.0,2019.0,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
264,2019-12-27,0.000,68.5,360,cbd,7.0,5.0,0.0,2019.0,27.0,12.0
265,2019-12-28,0.000,73.5,361,cbd,8.0,5.0,0.0,2019.0,28.0,12.0
266,2019-12-29,0.000,77.0,362,cbd,7.0,4.0,0.0,2019.0,29.0,11.0
267,2019-12-30,0.000,87.0,363,cbd,5.0,3.0,0.0,2019.0,30.0,12.0


In [33]:
# Displaying the column names of the final data
data.columns

Index(['Unnamed: 0', 'PRCP', 'TAVG', 'index', 'Cbd_Regional', 'total', 'left',
       'left_avg', 'year', 'day', 'hour'],
      dtype='object')

In [34]:
# Data changes
data = data.rename(columns={"Unnamed: 0": "Date"})    # renaming the date column name
X = data[['PRCP','TAVG','Cbd_Regional']]              # Selecting the weather attribute and region attribute as input
X.reset_index(drop=True, inplace=True)               # setting the index

In [35]:
# Encoding the categorical values
X['Cbd_Regional']=X['Cbd_Regional'].astype('category')
X['Cbd_Regional'] = X['Cbd_Regional'].cat.codes

<ipython-input-35-060556ec86c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cbd_Regional']=X['Cbd_Regional'].astype('category')
<ipython-input-35-060556ec86c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cbd_Regional'] = X['Cbd_Regional'].cat.codes


In [36]:
# Target variable
y = data[['total']]

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=0.40, random_state=42)

In [38]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [51]:
# Model prediction - preciptation-1.5,average temperature - 65, cbd - 0
reg.predict(np.array([[1.5, 65, 0]]))

array([[9.40392852]])

In [40]:
# Predicting the values
y_pred = reg.predict(X_test)

In [41]:
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [42]:
# Results
regression_results(y_test, y_pred)

explained_variance:  0.0052
mean_squared_log_error:  0.2944
r2:  0.0038
MAE:  4.5768
MSE:  37.2835
RMSE:  6.106


In [43]:
# Random forest reg
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(max_depth=2, random_state=0,n_estimators = 50)
rf_reg.fit(X_train, y_train.values.ravel())

RandomForestRegressor(max_depth=2, n_estimators=50, random_state=0)

In [44]:
import joblib
joblib.dump(reg, 'rfmodel2.dump')

['rfmodel2.dump']

In [49]:
'''manual_test = [[1.5, 75, 0],
              [1.5, 75, 1]]

'''
rf_reg.predict(np.array([[1.5, 65, 1]]))
#y_pred = rf_reg.predict(manual_test)
#y_pred


array([9.32836416])

In [47]:
# Predicting the values
y_pred = rf_reg.predict(X_test)

In [ ]:
# Results
regression_results(y_test, y_pred)

In [ ]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=1.0)
reg.fit(X_train, y_train)

In [ ]:
# Predicting the values
y_pred = reg.predict(X_test)

In [ ]:
# Results
regression_results(y_test, y_pred)

In [ ]:
display(X_train)